In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/SIR/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files['대구']
data

,stdDay,Susceptible,Alpha_Infected,Alpha_Recovered,Alpha_Dead,Alpha_alpha,Alpha_beta,Alpha_gamma,Delta_Infected,Delta_Recovered,Delta_Dead,Delta_alpha,Delta_beta,Delta_gamma
0,2021-04-12,2380065,39.65,163.36,2.00,0.059242,0.058260,0.005296,0.32,0.00,0.00,0.564647,0.000000,0.0
1,2021-04-13,2380056,39.47,165.67,2.21,0.072737,0.079807,0.000000,0.50,0.00,0.00,0.441681,0.000000,0.0
2,2021-04-14,2380045,39.18,168.82,2.21,0.119906,0.037519,0.000000,0.72,0.00,0.00,0.501913,0.000000,0.0
3,2021-04-15,2380027,42.39,170.29,2.21,0.024628,0.084218,0.000000,1.08,0.00,0.00,0.074358,0.000000,0.0
4,2021-04-16,2380023,39.86,173.86,2.21,0.078574,0.052684,0.000000,1.16,0.00,0.00,0.207690,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-08-11,2376752,12.67,486.64,3.05,0.030942,0.132597,0.000000,814.69,726.09,0.84,0.045715,0.052928,0.0
122,2021-08-12,2376713,11.38,488.32,3.05,0.050350,0.163445,0.000000,808.62,769.21,0.84,0.067316,0.059039,0.0
123,2021-08-13,2376656,10.09,490.18,3.05,0.081696,0.136769,0.000000,815.03,816.95,0.84,0.096082,0.043459,0.0
124,2021-08-14,2376574,9.53,491.56,3.05,0.060128,0.072403,0.000000,857.51,852.37,0.84,0.063482,0.020653,0.0


In [3]:
learning_rates_list = [1e-3, 1e-4, 1e-5]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4, 8]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout]
                        i += 1
                        
print(i)

144


In [4]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'Alpha_alpha')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [18]:
best_model = {}
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 50, 60, 1, batch_size, 'mto')
        
        if num_model == 0:
            model = RNN(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM(input_size = input_size,
                         hidden_size = hidden_size,
                         sequence_length = sequence_length,
                         num_layers = num_layers, 
                         dropout = dropout, 
                         device = device).to(device)
        
        elif num_model == 2:
            model = GRU(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 4:
            model = BiLSTM(input_size = input_size,
                           hidden_size = hidden_size,
                           sequence_length = sequence_length,
                           num_layers = num_layers, 
                           dropout = dropout, 
                           device = device).to(device)

        elif num_model == 5:
            model = BiGRU(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                      test_loader, 
                                                      model, 
                                                      criterion, 
                                                      optimizer, 
                                                      num_epochs, 
                                                      patience,
                                                      device)
    
        label_y, predicted = predict_mto(model, df, x_ss, y_ms)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout]

대구_RNN:   0%|          | 0/144 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
best_model

{'RNN': [0.17135245369187363,
  RNN(
    (rnn): RNN(3, 32, num_layers=4, batch_first=True, dropout=0.25)
    (fc1): Linear(in_features=1920, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU()
  ),
  [0.010839464142918587,
   0.03235950693488121,
   0.001861018710769713,
   0.005299543961882591,
   0.003218377474695444,
   0.0016474038129672408,
   0.0024449056945741177,
   0.002643004758283496,
   0.001988958101719618,
   0.0017009811708703637,
   0.0017400338547304273,
   0.001983668887987733,
   0.0019829177763313055,
   0.0018117482541128993,
   0.0016259641852229834,
   0.0016819507582113147,
   0.001821400597691536,
   0.0017966128652915359,
   0.0016767034539952874,
   0.0016646706499159336,
   0.001671552425250411,
   0.001638692687265575,
   0.0016474940348416567,
   0.001649720361456275,
   0.0016715730307623744,
   0.0014358299085870385,
   0.00137556227855

In [7]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/SIR/alpha/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/SIR/alpha/{key}.pkl")